In [24]:
# IMPORTS

import pandas as pd
import numpy as np
import re
import pprint
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk import ne_chunk, pos_tag
nltk.download('words')
from nltk.corpus import words as nltk_words
import allennlp
import six
from itertools import chain



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Y\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Y\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Sex in the city dataset

In [17]:
# LOAD SEX IN THE CITY

df = pd.read_csv(r'C:\Users\Y\Documents\MILA\Gender_Generalization\Data\SATC_all_lines.csv')
data = df['Line'].tolist()
print(data[:10])
print(type(data[0]))

['Once upon a time, an English journalist came to New York.', 'Elizabeth was attractive and bright.', "Right away she hooked up with one of the city's typically eligible bachelors.", 'The question remains-- Is this really a company we want to own? ', 'Tim was 42, a well-liked and respected investment banker who made about two million a year.', 'They met one evening, in typical New York fashion at a gallery opening.', 'Like it? ', "Yes, actually. I think it's quite interesting. What?", ' I feel like I know you from somewhere.', 'Doubtful. I only just moved here from London.']
<class 'str'>


In [26]:
# VALIDATE IF ALL df['Line'] = STRING & TOKENIZE SENTENCE
sentences = []
for s in data:
    if isinstance(s, six.string_types):
        sent = sent_tokenize(s)
        sentences.append(sent)

print (sentences[:10])
print(type(sentences))

[['Once upon a time, an English journalist came to New York.'], ['Elizabeth was attractive and bright.'], ["Right away she hooked up with one of the city's typically eligible bachelors."], ['The question remains-- Is this really a company we want to own?'], ['Tim was 42, a well-liked and respected investment banker who made about two million a year.'], ['They met one evening, in typical New York fashion at a gallery opening.'], ['Like it?'], ['Yes, actually.', "I think it's quite interesting.", 'What?'], [' I feel like I know you from somewhere.'], ['Doubtful.', 'I only just moved here from London.']]
<class 'list'>


In [27]:
# MAKE ONE LIST AND NOT NESTED BEFORE PREPROCESSING
flat_list = []
for sublist in sentences:
    for item in sublist:
        flat_list.append(item)
        
print(flat_list[:20])

['Once upon a time, an English journalist came to New York.', 'Elizabeth was attractive and bright.', "Right away she hooked up with one of the city's typically eligible bachelors.", 'The question remains-- Is this really a company we want to own?', 'Tim was 42, a well-liked and respected investment banker who made about two million a year.', 'They met one evening, in typical New York fashion at a gallery opening.', 'Like it?', 'Yes, actually.', "I think it's quite interesting.", 'What?', ' I feel like I know you from somewhere.', 'Doubtful.', 'I only just moved here from London.', 'London?', 'Really?', "That's my all-time favorite city.", '- It is?', 'Absolutely.', 'It was love at first sight.', 'You know I think perhaps I have met you somewhere before.']


In [34]:
# FINAL PREPROCESS FOR SEX IN THE CITY
clean_sent = preprocess(flat_list)

print(clean_sent[600:700])
sex_and_city = makecsv(clean_sent)

['Let me get the door for you, miss.', "I was hoping you'd stay over.", "Look, Sean I don't think this is gonna work.", "You obviously want to get married and I don't know what I want.", 'You could smell my desperation, right?', "It's just I was trying you on.", 'All I hear is I want to get married.', 'And none of you says yes.', "I'm so tired of going through women.", 'I just want to get married.', "I may know someone who's perfect for you.", 'They started out casual a brunch here, a concert there.', 'But pretty soon they were visiting china.', 'I always wanted this pattern for formal dining.', 'Charlotte broke it off then and there.', 'He was American Classic; she was French Country.', 'Your friend never called like she promised.', 'As I sifted through the rubble of my marriage skirmish I had a thought.', 'Maybe the fight between marrieds and singles is like the war in Northern lreland.', "We're all basically the same but somehow we wound up on different sides.", "Sure, it'd be great

# Preprocessing functions

In [30]:
# PREPROCESSING FUNCTION
def preprocess(sentences):
    clean_sentences = []
    for s in sentences:
        new_s = re.sub(r'\n+', ' ', s)                  # remove breakline
        #new_s = re.sub(r'\([^)]*\)',' ',new_s)
        new_s = re.sub(r'[^\w\/,:;.?\\\']', ' ', new_s) # removes all weird characters except punctuations
        new_s = new_s.replace("//", "")                 # remove quotation marks
        new_s = re.sub(r'\d+.', '',new_s)               # removes number followed by a .
        new_s = re.sub(r'/\s\s+/g', ' ', new_s)     
        new_s = new_s.replace('/', '')                  # removes forward slash
        new_s = new_s.replace('_', '')
        new_s = re.sub(r' +',' ',new_s)                 # removes extra spaces 
        new_s = new_s.lstrip()                          # removes spaces at the beginning of a string
        new_s = new_s.rstrip()                          # removes spaces at the end of a string
        wordcount = len(new_s.split())
        if new_s !='' and wordcount > 5:                 # sentences < 5 words are rejected
            clean_sentences.append(new_s)
    return(clean_sentences) 

# Make into .csv file

In [33]:
# CSV
def makecsv(sent_list):
    df = pd.DataFrame(sent_list)
    df.to_csv("Ready_2_Filter", header=False,index=None)
    return df